# Лабораторная работа 4. Разведочный анализ данных

## Часть 1. Описательный анализ.

Скачайте в рабочую директорию файл `cardio_train.csv` - в нем находятся данные, с которыми вы будете работать.

Данная работа состоит из **трех** заданий. Они записаны в блокноте после символов 📌📌📌 . Для записи ответов используйте ячейки под заданиями, содержащие запись `# Ваш код`.

При выполнении работы рекомендуется обращаться:

- [к справке по Python 3](https://docs.python.org/3/tutorial/index.html),
- к учебнику `У. Маккини. Python и анализ данных`
- [к шпаргалке по Pandas на Harbr](https://habr.com/ru/company/ruvds/blog/494720/)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Полезный функционал: lambda-фукции, метод pandas.apply()

Анонимные или лямбда-функции - это простые однострочные функции, возвращающие значение. Определяются они с помощью слова `lambda`, которое указывает, что далее будет записан аргумент, а затем - тело анонимной функции. В примере ниже покоординатное применения функции f(x) реализовано двумя разными способами: обычным способом и с помощью лямбда функции.

In [2]:
def apply_to_list(mylist, myfun):
        return([myfun(x) for x in mylist])

In [3]:
# обычное определение пользовательской функции
def f(x):
    return(3*x**2+3)
apply_to_list([1,2,3], f)

[6, 15, 30]

In [4]:
# лямбда-функция
apply_to_list([1,2,3], lambda x:3*x**2+3)

[6, 15, 30]

Часто встречается ситуация, когда функция, определенная для скалярной переменной или одномерного вектора,  должна быть применена к столбцу или строке датафрейма.
В этом случае можно воспользоваться методом `apply` объекта DataFrame.
Разберитесь, что происходит в ячейках ниже.

In [5]:
my_df=pd.DataFrame(data={"a":[1,2,3,1,2,2], "b":[1.25, 3.83, 2.789,3.456, 6.45, 3], "name":["Mary", "Kate", "Olga", "Kate", "Olga", "Kate"]})

In [6]:
my_df['a'].apply(f)

0     6
1    15
2    30
3     6
4    15
5    15
Name: a, dtype: int64

In [7]:
my_df[['a','b']].apply(sum)

a    11.000
b    20.775
dtype: float64

In [8]:
my_df[['a','b']].apply(sum, axis='columns')

0    2.250
1    5.830
2    5.789
3    4.456
4    8.450
5    5.000
dtype: float64

In [9]:
my_df['b'].apply(int)

0    1
1    3
2    2
3    3
4    6
5    3
Name: b, dtype: int64

**Замечание:** Отметим, что очень удобно использовать метод `apply` для изменения типа столбца датафрейма.

## Агрегирование данных
Рассмотрим несколько инструментов агрегирования данных:
- **функция groupby**

  позволяет группировать данные по некоторому признаку и применять к результату различные функции.

Изучите представленные ниже примеры применения функции `groupby` и прочитайте более подробную информацию о этой функции в книге `У. Маккини. Python и анализ данных, стр. 312-315 `

In [10]:
my_df.groupby('name')['a'].min()

name
Kate    1
Mary    1
Olga    2
Name: a, dtype: int64

In [11]:
my_df.groupby(['name','a'])['b'].mean()

name  a
Kate  1    3.456
      2    3.415
Mary  1    1.250
Olga  2    6.450
      3    2.789
Name: b, dtype: float64

- **сводная таблица pivot_table**

Это средство обобщения данных. Оно агрегирует таблицу по одному или нескольким ключам и строит другую таблицу, в кторой одни групповые ключи расположены в строках, а другие - в столбцах.

Изучите примеры применения метода `pivot_table` объекта DataFrame и прочитайте более подробно о сводных таблицах в учебнике `У. Маккини. Python и анализ данных, стр. 331-333 `

In [12]:
my_df.pivot_table(values='b', index='name', columns='a' )
# если не задавать фунцию аггрегирования aggfunc, то по умолчанию используется mean

a,1,2,3
name,,,
Kate,3.456,3.415,NaN
Mary,1.250,NaN,NaN
Olga,NaN,6.450,2.789


In [13]:
my_df.pivot_table(values='b', index='name', columns='a', fill_value=0 )

a,1,2,3
name,,,
Kate,3.456,3.415,0.000
Mary,1.250,0.000,0.000
Olga,0.000,6.450,2.789


In [14]:
my_df.pivot_table(values='b', index='name', columns='a', aggfunc='count',  fill_value=0 )

a,1,2,3
name,,,
Kate,1,2,0
Mary,1,0,0
Olga,0,1,1


- **таблица сопряженности crosstab**

это частный случай сводной таблицы, в которой представлены групповые частоты. Более подробную информацию прочтите в учебнике  `У. Маккини. Python и анализ данных, стр. 334 `, а также в справке по этому методу https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html

In [15]:
pd.crosstab(my_df.a, my_df.name)

name,Kate,Mary,Olga
a,,,
1,1,1,0
2,2,0,1
3,0,0,1


## Работа с набором данных о сердечно-сосудистых заболеваниях (ССЗ)
Набор данных **Cardiovascular Disease dataset** содержит следующую информацию о пациентах:
- age (возраст, в днях)
- height (рост, в см.)
- weight (вес, в кг.)
- gender (пол)
- ap_hi (систолическое давление)
- ap_lo (диастолическое давление)
- cholesterol  (уровень холестерина:  1: нормальный, 2: повышенный, 3: очень высокий)
- gluc (уровень глюкозы: 1: нормальный, 2: повышенный, 3: очень высокий)
- smoke(курение)
- alco (употребление алкоголя)
- active (физическая активность)
- cardio (отсутствие или наличие ССЗ)

📌📌📌 **Задание 1**

1. Импортируйте данные из файла `cardio_train.csv`. Определите, есть ли в полученном датасете категориальные или бинарные признаки. Если да, то для таких признаков выведите на экран: 
   - число наблюдений без пропусков
   - число уникальных значений
   - самое частое значение
   - частота, с которой встречается значение top
   
    Для остальных признаков выведите:
   - count - число наблюдений без пропусков
    - mean - среднее значение
    - std - стандартное отклонение
    - min - минимум
    - 50% - медиана
    - 25% - 25% квантиль
    - 75% - 75% квантиль
    - max - максимум

    Для выполнения этого задания воспользуйтесь методом `pandas.describe()` 


2. Выведите частоты встречаемости каждого из значений бинарных признаков

In [16]:
# ваш код
df = pd.read_csv('cardio_train.csv', sep=';')

categorical_features = ['gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio']
binary_features = ['gender','smoke', 'alco', 'active', 'cardio'] 
df[categorical_features] = df[categorical_features].astype(str)

# Вывод информации о категориальных  признаках
print("Категориальные и бинарные признаки:")
cat_stats = df.describe(include='object')
print(cat_stats)

# Вывод описательных статистик для числовых признаков
numerical_features = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
print("\nЧисловые признаки:")
num_stats = df[numerical_features].describe()
print(num_stats)

# Вывод частот для бинарных признаков
print("\nЧастоты бинарных признаков:")
for feature in binary_features:
    print(f"\n{feature}:")
    print(df[feature].value_counts())

Категориальные и бинарные признаки:
       gender cholesterol   gluc  smoke   alco active cardio
count   70000       70000  70000  70000  70000  70000  70000
unique      2           3      3      2      2      2      2
top         1           1      1      0      0      1      0
freq    45530       52385  59479  63831  66236  56261  35021

Числовые признаки:
                age        height        weight         ap_hi         ap_lo
count  70000.000000  70000.000000  70000.000000  70000.000000  70000.000000
mean   19468.865814    164.359229     74.205690    128.817286     96.630414
std     2467.251667      8.210126     14.395757    154.011419    188.472530
min    10798.000000     55.000000     10.000000   -150.000000    -70.000000
25%    17664.000000    159.000000     65.000000    120.000000     80.000000
50%    19703.000000    165.000000     72.000000    120.000000     80.000000
75%    21327.000000    170.000000     82.000000    140.000000     90.000000
max    23713.000000    250.0000

📌📌📌 **Задание 2**

Создайте подходящие сводную таблицу (pivot_table) и таблицу сопряженности (crosstab) и предварительно ответьте с помощью них на следующие вопросы:

**вариант 1**:
- верно ли, что некурящие люди, страдающие ССЗ, в среднем имеют более высокий вес, чем остальные? 
- верно ли, что среди курящих процент страдающих ССЗ больше?

**вариант 2**:
- верно ли, что  с повышением уровнем холестерина при одинаковом уровне глюкозы систолическое давление у пациентов в среднем становится выше? 
- верно ли, что курящие люди больше склонны употреблять алкоголь?

**вариант 3**:
- верно ли, что повышение уровня глюкозы и курение негативно влияют на систолическое давление? 
- верно ли, что среди физически активных людей процент страдающих ССЗ больше?

**вариант 4**:
- каковы минимальные, максимальные и средние значения разности между систолическим и диастолическим давлением для больных ССЗ и здоровых людей?  
- верно ли, что среди мужчин процент страдающих ССЗ больше? (P.S. А кто-же из наших пациентов - мужчины? Как бы это выяснить по данным?)

In [17]:
# ваш код
# 1. Разность давлений для больных и здоровых
df['pressure_diff'] = df['ap_hi'] - df['ap_lo']
pressure_stats = df.groupby('cardio')['pressure_diff'].agg(['min', 'max', 'mean'])
print("Статистики разности давлений:")
print(pressure_stats)

# 2. Процент страдающих ССЗ среди мужчин и женщин

mean_height_by_gender = df.groupby('gender')['height'].mean()
print('\n соотношение gender и среднего роста')
print(mean_height_by_gender)
# 1 - женщины
cross_tab = pd.crosstab(df['gender'], df['cardio'],normalize='index')
print("\nПроцент страдающих ССЗ по полу:")
print(cross_tab)

Статистики разности давлений:
          min    max       mean
cardio                         
0       -9670  13930  36.180834
1      -10800  15940  28.188113

 соотношение gender и среднего роста
gender
1    161.355612
2    169.947895
Name: height, dtype: float64

Процент страдающих ССЗ по полу:
cardio         0         1
gender                    
1       0.503273  0.496727
2       0.494769  0.505231
